## Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

#### Instructions: 
    
For this assignment, you will be required to explore and cluster the neighborhoods in Toronto.
1. Start by creating a new Notebook for this assignment.
2. Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below:

| | PostalCode | Borough | Neighborhood |
|--|-----|------------------|--------------------|
| 0| M5G | Downtown Toronto | Central Bay Street |
| 1| M2H | North York | Hillcrest Village |
| 2| M4B | East York | Parkview Hill, Woodbine Gardens |
| 3| M1J | Scarborough | Scarborough Village |
| 4| M4G | East York | Leaside |
| 5| M4M | East Toronto | Studio District |
| 6| M1R | Scarborough | Wedford, Maryvale |
| 7| M9V | Etobicoke | South Steeles, Silverstone, Humbergate, Jamest...|
| 8| M9L | North York | Humber Summit |
| 9| M5V | Downtown Toronto | CN Tower, King and Spadina, Railway Lands, Har... |
|10| M1B | Scarborough | Malvern, Rouge |
|11| M5V | Downtown Toronto | Regent Park, Harbourfront |

3. To create the above dataframe:

    * The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
    * Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
    * More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
    * If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
    * Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
    * In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

4. Submit a link to your Notebook on your Github repository. (10 marks)

Install and import packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
!pip install beautifulsoup4

In [3]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [4]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html = urlopen(url)

In [5]:
soup = BeautifulSoup(html, 'html.parser')
type(soup)

bs4.BeautifulSoup

In [6]:
soup.title

<title>List of postal codes of Canada: M - Wikipedia</title>

In [7]:
rows = soup.find_all('tr')
print(rows[:10])

[<tr>
<th>Postal Code
</th>
<th>Borough
</th>
<th>Neighborhood
</th></tr>, <tr>
<td>M1A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>, <tr>
<td>M2A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>, <tr>
<td>M3A
</td>
<td>North York
</td>
<td>Parkwoods
</td></tr>, <tr>
<td>M4A
</td>
<td>North York
</td>
<td>Victoria Village
</td></tr>, <tr>
<td>M5A
</td>
<td>Downtown Toronto
</td>
<td>Regent Park, Harbourfront
</td></tr>, <tr>
<td>M6A
</td>
<td>North York
</td>
<td>Lawrence Manor, Lawrence Heights
</td></tr>, <tr>
<td>M7A
</td>
<td>Downtown Toronto
</td>
<td>Queen's Park, Ontario Provincial Government
</td></tr>, <tr>
<td>M8A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>, <tr>
<td>M9A
</td>
<td>Etobicoke
</td>
<td>Islington Avenue, Humber Valley Village
</td></tr>]


In [8]:
for row in rows:
    row_td = row.find_all('td')
print(row_td)
type(row_td)

[<td align="center" style="border: 1px solid #FF0000; background-color: #FFE0E0; font-size: 135%;" width="5%"><a href="/wiki/List_of_postal_codes_of_Canada:_A" title="List of postal codes of Canada: A">A</a>
</td>, <td align="center" style="border: 1px solid #FF4000; background-color: #FFE8E0; font-size: 135%;" width="5%"><a href="/wiki/List_of_postal_codes_of_Canada:_B" title="List of postal codes of Canada: B">B</a>
</td>, <td align="center" style="border: 1px solid #FF8000; background-color: #FFF0E0; font-size: 135%;" width="5%"><a href="/wiki/List_of_postal_codes_of_Canada:_C" title="List of postal codes of Canada: C">C</a>
</td>, <td align="center" style="border: 1px solid #FFC000; background-color: #FFF8E0; font-size: 135%;" width="5%"><a href="/wiki/List_of_postal_codes_of_Canada:_E" title="List of postal codes of Canada: E">E</a>
</td>, <td align="center" style="border: 1px solid #FFFF00; background-color: #FFFFE0; font-size: 135%;" width="5%"><a href="/wiki/List_of_postal_code

bs4.element.ResultSet

In [9]:
import re

list_rows = []
for row in rows:
    cells = row.find_all('td')
    str_cells = str(cells)
    clean = re.compile('<.*?>')
    clean2 = (re.sub(clean, '',str_cells))
    list_rows.append(clean2)
print(clean2)
type(clean2)

[A
, B
, C
, E
, G
, H
, J
, K
, L
, M
, N
, P
, R
, S
, T
, V
, X
, Y
]


str

In [10]:
df = pd.DataFrame(list_rows)
df.head(10)

,0
0,[]
1,"[M1A\n, Not assigned\n, Not assigned\n]"
2,"[M2A\n, Not assigned\n, Not assigned\n]"
3,"[M3A\n, North York\n, Parkwoods\n]"
4,"[M4A\n, North York\n, Victoria Village\n]"
5,"[M5A\n, Downtown Toronto\n, Regent Park, Harbo..."
6,"[M6A\n, North York\n, Lawrence Manor, Lawrence..."
7,"[M7A\n, Downtown Toronto\n, Queen's Park, Onta..."
8,"[M8A\n, Not assigned\n, Not assigned\n]"
9,"[M9A\n, Etobicoke\n, Islington Avenue, Humber ..."


In [11]:
df.tail(10)

,0
175,"[M4Z\n, Not assigned\n, Not assigned\n]"
176,"[M5Z\n, Not assigned\n, Not assigned\n]"
177,"[M6Z\n, Not assigned\n, Not assigned\n]"
178,"[M7Z\n, Not assigned\n, Not assigned\n]"
179,"[M8Z\n, Etobicoke\n, Mimico NW, The Queensway ..."
180,"[M9Z\n, Not assigned\n, Not assigned\n]"
181,"[\n, \n]"
182,[\n\n\nNL\n\nNS\n\nPE\n\nNB\n\nQC\n\nON\n\nMB\...
183,"[NL\n, NS\n, PE\n, NB\n, QC\n, ON\n, MB\n, SK\..."
184,"[A\n, B\n, C\n, E\n, G\n, H\n, J\n, K\n, L\n, ..."


In [12]:
df1 = df[0].str.split(',', 2, expand = True)
df1.head(10)

,0,1,2
0,[],None,None
1,[M1A\n,Not assigned\n,Not assigned\n]
2,[M2A\n,Not assigned\n,Not assigned\n]
3,[M3A\n,North York\n,Parkwoods\n]
4,[M4A\n,North York\n,Victoria Village\n]
5,[M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n]"
6,[M6A\n,North York\n,"Lawrence Manor, Lawrence Heights\n]"
7,[M7A\n,Downtown Toronto\n,"Queen's Park, Ontario Provincial Government\n]"
8,[M8A\n,Not assigned\n,Not assigned\n]
9,[M9A\n,Etobicoke\n,"Islington Avenue, Humber Valley Village\n]"


In [13]:
df1 = df1.loc[1:180,:]
df1.reset_index(drop = True)
df1.head(10)
df1.tail(10)

,0,1,2
171,[M9Y\n,Not assigned\n,Not assigned\n]
172,[M1Z\n,Not assigned\n,Not assigned\n]
173,[M2Z\n,Not assigned\n,Not assigned\n]
174,[M3Z\n,Not assigned\n,Not assigned\n]
175,[M4Z\n,Not assigned\n,Not assigned\n]
176,[M5Z\n,Not assigned\n,Not assigned\n]
177,[M6Z\n,Not assigned\n,Not assigned\n]
178,[M7Z\n,Not assigned\n,Not assigned\n]
179,[M8Z\n,Etobicoke\n,"Mimico NW, The Queensway West, South of Bloor..."
180,[M9Z\n,Not assigned\n,Not assigned\n]


In [14]:
df1[0] = df1[0].str.strip('[')
df1[0] = df1[0].str.strip(']')
df1[0] = df1[0].str.strip('\n')
df1[1] = df1[1].str.strip('\n')
df1[1] = df1[1].str.strip('')
df1[2] = df1[2].str.strip('\n]')
df1[2] = df1[2].str.strip('')

df1.head(10)

,0,1,2
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,"Regent Park, Harbourfront"
6,M6A,North York,"Lawrence Manor, Lawrence Heights"
7,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M8A,Not assigned,Not assigned
9,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
10,M1B,Scarborough,"Malvern, Rouge"


In [15]:
df1.rename(columns={0:'PostalCode', 1:'Borough', 2:'Neighborhood'}, inplace = True)
df1.head()

,PostalCode,Borough,Neighborhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [16]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180 entries, 1 to 180
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   PostalCode    180 non-null    object
 1   Borough       180 non-null    object
 2   Neighborhood  180 non-null    object
dtypes: object(3)
memory usage: 4.3+ KB


In [24]:
df1.loc[(df1['Borough'] == " Not assigned")]

,PostalCode,Borough,Neighborhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
8,M8A,Not assigned,Not assigned
11,M2B,Not assigned,Not assigned
16,M7B,Not assigned,Not assigned
...,...,...,...
175,M4Z,Not assigned,Not assigned
176,M5Z,Not assigned,Not assigned
177,M6Z,Not assigned,Not assigned
178,M7Z,Not assigned,Not assigned


In [22]:
df1.loc[(df1['Borough'] != " Not assigned") & (df1['Neighborhood'] == " Not assigned")]

,PostalCode,Borough,Neighborhood


There is no rows where Borough is assigned but neighborhood is not, so I only need to drop out rows where borough is not assigned.

In [17]:
df_final = df1.loc[df1['Borough'] != " Not assigned"]
df_final.head()

,PostalCode,Borough,Neighborhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,"Regent Park, Harbourfront"
6,M6A,North York,"Lawrence Manor, Lawrence Heights"
7,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [18]:
df_final.reset_index(drop = True)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South ..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, H..."


In [20]:
df_final.shape

(103, 3)